# 인터파크 자동 티켓팅

In [2]:
from selenium import webdriver

In [7]:
import json

In [3]:
driver = webdriver.Chrome()
driver.get("https://ticket.interpark.com")

In [5]:
%%writefile interpark.json
{
    "id":"gmltn4993",
    "pw":"~khs7518536"
}

Overwriting interpark.json


In [8]:
with open("interpark.json",encoding='utf8') as fp:
    account = json.load(fp)

In [6]:
_login = [_["name"] for _ in driver.get_cookies()]

In [7]:
_logout = [_["name"] for _ in driver.get_cookies()]

여기서 로그인했을때와 로그아웃했을 때 세션의 차이를 봤을 때 ISession%5FID의 유무가 있음

In [17]:
[_["name"] for _ in driver.get_cookies()]

['GNBBookCnt',
 'MemHash',
 'SAVEID',
 'TSecLogin',
 'memRealName',
 'HMem%5FNO',
 'interparkstamp',
 'IEGS%5FGUIF',
 'TSession%5FID',
 'IMem%5FNO',
 'iaf',
 'egsTp',
 'pcid',
 'interparkSNO',
 'ISession%5FID',
 'TKLogin',
 'tempinterparkGUEST',
 'OAX',
 'TPartner',
 'alMemNm',
 '_gid',
 'ASPSESSIONIDSACTDRRQ',
 '_ga',
 'PrivacyCertify',
 '_trs_sid',
 'captcha_text',
 '_fbp',
 'siteAgree',
 '_trs_id',
 'cto_lwid',
 'pwdChgYn',
 'crn',
 'ASPSESSIONIDSCCTDSRQ',
 'ASPSESSIONIDACBRARSS',
 'interparkID',
 'TMem%5FNO',
 'authRtnUrl',
 '_trs_flow']

In [9]:
#  로그인 버튼 찾기
def login(account, method="login"):
    try:
        login = driver.find_element_by_css_selector("#logstatus > a")
        if method == "login":
            if login and login.get_attribute("id") == "aLogin":
                driver.get(login.get_attribute("href"))
                driver.find_element_by_css_selector("#UID").send_keys(account["id"])
                driver.find_element_by_css_selector("#PWD").send_keys(account["pw"])
                driver.find_element_by_css_selector("div.loginBtn > button").click()
        else:
            login.click()
    except:
        return False
    return True

In [10]:
if "ISession%5FID" not in [_["name"] for _ in driver.get_cookies()]:
    login(account)
else:
    login(None,"logout")
# 세션을 확인해서 로그인 되어있으면로그아웃을 
# 로그아웃 되어있으면 로그인을 시킨다.

In [12]:
login(account,"logout")

True

In [14]:
login(account)

True

In [11]:
def gnbbtns(category="콘서트"):
    links = [_ for _ in driver.find_elements_by_css_selector("ul.gnbBtns a[class^=btn]")]
    
    titles = [_.find_element_by_css_selector("img").get_attribute("alt") for _ in links]

    if category in titles:
        return links[titles.index(category)].get_attribute("href")
    return False

In [9]:
gnbbtns()
#기본적으로 콘서트

'http://ticket.interpark.com/ConcertIndex.asp'

In [12]:
gnbbtns("뮤지컬")
#뮤지컬로 이동됨

'http://ticket.interpark.com/MusicalIndex.asp'

In [12]:
url = gnbbtns("콘서트")
if url:
    driver.get(url)

In [13]:
def hotissue(issue=""):
    issues = [_ for _ in driver.find_elements_by_css_selector("div.wrap_issue dl.list >dt.issue_obj > a")]
    title = [_.find_element_by_css_selector("span.txt1").text for _ in issues]
    
    if issue in title:
        return issues[title.index(issue)].get_attribute("href")
    else:
        return issues[0].get_attribute("href")

In [12]:
hotissue("박정현")

'http://ticket.interpark.com/Ticket/Goods/TPBridge.asp?GoodsCode=19008643'

In [14]:
driver.get(hotissue("박정현"))
# 해당하는 가수가 없으면 첫번째꺼 issue[0]을 선택한다.

In [15]:
def bookWindow(options=None):
    if not options :
        driver.find_element_by_css_selector(".tk_dt_btn_TArea > a.btn_rev").click()

In [16]:
bookWindow()

In [17]:
driver.switch_to_window(driver.window_handles[-1])
#프레임 변경하기

c:\users\김엄마\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: use driver.switch_to.window instead
  """Entry point for launching an IPython kernel.


In [18]:
frame = driver.find_element_by_id('ifrmBookStep')

In [19]:
driver.switch_to_frame(frame)

c:\users\김엄마\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: use driver.switch_to.frame instead
  """Entry point for launching an IPython kernel.


In [20]:
script = driver.find_elements_by_id('CellPlayDate')[1].get_attribute('onclick')
#날짜 선택핳기

In [21]:
driver.execute_script(script)

In [22]:
driver.switch_to_default_content()
# ifram 이전의 프레임으로 가기

c:\users\김엄마\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: use driver.switch_to.default_content instead
  """Entry point for launching an IPython kernel.


In [23]:
driver.execute_script("javascript:fnNextStep('P');")

좌석선택

In [24]:
iframe1 = driver.find_element_by_id('ifrmSeat')
driver.switch_to_frame(iframe1)
iframe2 = driver.find_element_by_id('ifrmSeatDetail')
driver.switch_to_frame(iframe2)

c:\users\김엄마\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: use driver.switch_to.frame instead
  
c:\users\김엄마\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: use driver.switch_to.frame instead
  after removing the cwd from sys.path.


In [27]:
seat = driver.find_elements_by_css_selector('#TmgsTable > tbody > tr > td > img')[4]
script = seat.get_attribute('onclick')
driver.execute_script(script)

In [28]:
driver.switch_to_default_content()
iframe1 = driver.find_element_by_id('ifrmSeat')
driver.switch_to_frame(iframe1)
driver.execute_script('fnSelect();')

c:\users\김엄마\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: use driver.switch_to.default_content instead
  """Entry point for launching an IPython kernel.
c:\users\김엄마\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: use driver.switch_to.frame instead
  This is separate from the ipykernel package so we can avoid doing imports until


In [29]:
driver.switch_to_window(driver.window_handles[-1])
driver.switch_to_default_content()
iframe = driver.find_element_by_id('ifrmBookStep')
driver.switch_to_frame(iframe)
driver.find_element_by_xpath('/html/body/div/div[1]/div/table[1]/tbody/tr/td/table/tbody/tr/td[3]/select/option[2]').click()

c:\users\김엄마\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: use driver.switch_to.window instead
  """Entry point for launching an IPython kernel.
c:\users\김엄마\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: use driver.switch_to.default_content instead
  
c:\users\김엄마\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: use driver.switch_to.frame instead
  after removing the cwd from sys.path.


In [34]:
driver.switch_to_default_content()
driver.execute_script("javascript:fnNextStep('P');")

c:\users\김엄마\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: use driver.switch_to.default_content instead
  """Entry point for launching an IPython kernel.


In [35]:
iframe = driver.find_element_by_id('ifrmBookStep')
driver.switch_to_frame(iframe)

c:\users\김엄마\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: use driver.switch_to.frame instead
  


In [39]:
date = driver.find_element_by_id('YYMMDD')
date.send_keys('960915')

In [40]:
driver.switch_to_default_content()
driver.execute_script("javascript:fnNextStep('P');")

c:\users\김엄마\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: use driver.switch_to.default_content instead
  """Entry point for launching an IPython kernel.


In [42]:
frame = driver.find_element_by_xpath('//*[@id="ifrmBookStep"]')
driver.switch_to_frame(frame)

c:\users\김엄마\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: use driver.switch_to.frame instead
  


In [57]:
driver.find_element_by_xpath('//*[@id="BankCode"]/option[9]').click()

In [58]:
driver.switch_to_default_content()
driver.execute_script("javascript:fnNextStep('P');")

c:\users\김엄마\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: use driver.switch_to.default_content instead
  """Entry point for launching an IPython kernel.


In [61]:
iframe = driver.find_element_by_id('ifrmBookStep')
driver.switch_to_frame(iframe)

c:\users\김엄마\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: use driver.switch_to.frame instead
  


In [63]:
driver.find_element_by_xpath('//*[@id="CancelAgree"]').click()
driver.find_element_by_xpath('//*[@id="CancelAgree2"]').click()